In [ ]:
import base64

config_data = top_servers.iloc[0]["OpenVPN_ConfigData_Base64"]
with open("vpn_config.ovpn", "wb") as f:
    f.write(base64.b64decode(config_data))
print("OpenVPN configuration file 'vpn_config.ovpn' has been created.")

In [5]:
import requests
import pandas as pd
from io import StringIO

# Step 1: Fetch the VPN Gate CSV data
url = "https://www.vpngate.net/api/iphone/"

try:
    response = requests.get(url, timeout=10)
    raw_data = response.text

    # Step 2: Remove comment lines and take a small subset
    cleaned_data = "\n".join(
        line for line in raw_data.splitlines() if not line.startswith("*")
    )
    partial_data = "\n".join(cleaned_data.splitlines()[:30])  # first 20 rows

    # Step 3: Read CSV into DataFrame
    df = pd.read_csv(StringIO(partial_data), header=None)
    df.columns = [
        "HostName", "IP", "Score", "Ping", "Speed", "CountryLong", "CountryShort",
        "NumVpnSessions", "Uptime", "TotalUsers", "TotalTraffic", "LogType", "Operator",
        "Message", "OpenVPN_ConfigData_Base64"
    ]

    # Step 4: Convert numeric columns safely
    df["Ping"] = pd.to_numeric(df["Ping"], errors="coerce")
    df["NumVpnSessions"] = pd.to_numeric(df["NumVpnSessions"], errors="coerce")
    df["Speed"] = pd.to_numeric(df["Speed"], errors="coerce")

    # Drop rows with invalid ping
    df_clean = df.dropna(subset=["Ping", "NumVpnSessions", "Speed"])

    # Step 5: Sort by lowest Ping and usage
    top_servers = df_clean.sort_values(by=["Ping", "NumVpnSessions"], ascending=[True, True])

    # Step 6: Display result
    print(top_servers[["IP", "CountryLong", "Ping", "Speed", "NumVpnSessions", "Uptime"]])

except Exception as e:
    print("Error fetching or processing data:", str(e))


KeyboardInterrupt: 